<a href="https://colab.research.google.com/github/ashhwiithac22/Editable_Memory_Chatbot/blob/main/Editable_Memory_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from openai import OpenAI
import json
os.environ["OPENAI_API_KEY"] = "your-api-key-here"
client = OpenAI()
model = "gpt-4o-mini"

In [ ]:
#memory box
agent_memory = {"human":"", "agent":""}
system_prompt = ("You are a chatbot. You have a section of your context called [Memory] that contains information relevant to your conversation.")

In [ ]:
def core_memory_save(section:str, memory:str):
  agent_memory[section] += '\n' + memory

In [ ]:
core_memory_save_metadata = {
    "type": "function",
    "function":{
        "name": "core_memory_save",
        "description":"Save important information about you, the agent or human you are chatting with",
        "parameters":{
            "type":"object",
            "properties":{
                "section":{
                    "type":"string",
                    "enum":["agent", "human"],
                    "description": "must be either 'human' or 'agent'"
               },
                "memory": {
                    "type":"string",
                    "description": "memory to save in this section"
                }
            },
            "required":["section", "memory"]
        }

    }
}

In [ ]:
system_prompt_os = (system_prompt + '\n you must either call a tool (core_memory_save or write a response to the user)')

In [ ]:
def agent_step(user_message):

    messages = [
        {"role": "system", "content": system_prompt_os},
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        {"role": "user", "content": user_message},
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=[core_memory_save_metadata],
    )

    message = response.choices[0].message

    if message.tool_calls:
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)

        core_memory_save(arguments["section"], arguments["memory"])

        return "Memory saved"

    return message.content

In [ ]:
if __name__ == "__main__":
    print("Chatbot with memory started. Type 'exit' to stop\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            break

        response = agent_step(user_input)
        print("Bot:", response)

Chatbot with memory started. Type 'exit' to stop

